# Competição ML @SBS/DAA - 5ª Edição (2022/2023)

Esta competição está relacionada com o Trabalho Prático de Grupo das UCs do perfil Machine Learning: Fundamentos e Aplicações da Uminho e da UC Dados e Aprendizagem Automática, tendo, como destinatários, alunos do Mestrado em Engenharia Informática, do Mestrado em Matemática e Computação, e do Mestrado em Engenharia de Sistemas.

# Descrição da Competição
A previsão da quantidade de incidentes rodoviários é um conhecido problema de características estocásticas, não-lineares. Tem, contudo, aparecido na literatura um conjunto de modelos que demonstram um potencial assinalável neste tipo de previsões. Com isso em consideração, foi construído um dataset que contém dados referentes à quantidade e características dos incidentes rodoviários que ocorreram na cidade de Guimarães em 2021 (o dataset cobre um período que vai desde o dia 01 de Janeiro de 2021 até ao dia 31 de Dezembro do mesmo ano).

Com esta competição espera-se que os alunos desenvolvam e otimizem modelos de Machine Learning que sejam capazes de prever o número de incidentes rodoviários que irão acontecer na cidade de Guimarães a uma determinada hora.

## Carregar variaveis de ambiente

In [ ]:
!conda install python-dotenv -y  # Correr isto para instalar as dependencias necessarias

In [25]:
import os

from dotenv import load_dotenv

load_dotenv()

DATASET_SOURCE = os.environ.get("DATASET_SOURCE")

## Imports utilizados

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


% matplotlib inline

### Mudar o modo que a Interactive shell imprime as variáveis

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Loading the training dataset

In [26]:
train_df = pd.read_csv(os.path.basename(DATASET_SOURCE))

## Exploring

In [24]:
train_df.head()

train_df.describe()

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city_name           5000 non-null   object 
 1   magnitude_of_delay  5000 non-null   object 
 2   delay_in_seconds    5000 non-null   int64  
 3   affected_roads      4915 non-null   object 
 4   record_date         5000 non-null   object 
 5   luminosity          5000 non-null   object 
 6   avg_temperature     5000 non-null   float64
 7   avg_atm_pressure    5000 non-null   float64
 8   avg_humidity        5000 non-null   float64
 9   avg_wind_speed      5000 non-null   float64
 10  avg_precipitation   5000 non-null   float64
 11  avg_rain            5000 non-null   object 
 12  incidents           5000 non-null   object 
dtypes: float64(5), int64(1), object(7)
memory usage: 507.9+ KB


In [ ]:
for column in train_df.columns:
    train_df[column].value_counts()

In [ ]:
train_df[['magnitude_of_delay', 'delay_in_seconds']]

In [ ]:
sns.heatmap(train_df.corr(numeric_only=True))

In [ ]:
sns.displot(train_df['avg_rain'], kde=True)

In [ ]:
sns.histplot(train_df['avg_rain'])

In [ ]:
train_df.groupby(by=['avg_rain', 'incidents']).count()

## Data preparation

In [ ]:
X = train_df.drop(['city_name', 'magnitude_of_delay', 'affected_roads', 'luminosity', 'avg_rain', 'incidents'], axis=1)

In [ ]:
from sklearn import preprocessing

estimator = preprocessing.KBinsDiscretizer(n_bins=3, random_state=2021)

#X['delay_in_seconds_binned'] = pd.qcut(X['delay_in_seconds'], q=3)

In [ ]:
record_date = pd.DatetimeIndex(X['record_date'])

X['record_date_hour'] = record_date.hour
X['record_date_weekday'] = record_date.weekday

X.drop(columns=['record_date'], inplace=True)

X

In [ ]:
y = train_df['incidents']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

------

In [ ]:
# Criar bins (experimentar) -> avg temperature
train_df.loc[train_df['avg_temperature'].between(10, 19, 'both'), 'temperature'] = 'MEAN'
train_df.loc[train_df['avg_temperature'].between(0, 10, 'right'), 'grade'] = 'LOW'
x = train_df.loc[train_df['avg_temperature'].between(19, 35, 'right'), 'grade'] = 'HIGH'

In [ ]:
# Eliminar (initialmente) -> avg humidity
train_df.drop(['avg_humidity'], axis=1)

# Eliminar -> avg precipitation
train_df.drop(['avg_precipitation'], axis=1)

In [ ]:
sns.displot(train_df['avg_wind_speed'])

train_df['avg_wind_speed'].value_counts()

# Remover outliners -> avg wind speed
# outliers = train_df["avg_wind_speed"].quantile(8.0)

train_df[train_df["avg_wind_speed"] < 8.0]

### Model Fitting

In [ ]:
clf = DecisionTreeClassifier(random_state=2021)

clf.fit(X_train, y_train)

scores = cross_val_score(clf, X, y, cv=5)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, predictions)
# TP FP
# FN TN
disp = ConfusionMatrixDisplay(cm)

disp.plot()

In [ ]:
accuracy_score(y_test, predictions)
# (TP + TN) / (TP + FP + FN + TN)

precision_score(y_test, predictions, average='micro')
# TP / (TP + FP)

recall_score(y_test, predictions, average='micro')
# TP / (TP + FN)

In [ ]:
predictions

pd.DataFrame(predictions).to_csv("submission.csv")